In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'house-price-india-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4432988%2F7612597%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240414%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240414T130527Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D6174ed7ae9237117fd895836ca0a7fec740527e9bb42a9763c3d87119096b1e19f6eccc74efe794bb12ab472509bc9404e4c837b219aa388ac59ff8fa7c10feddae34b3bb995c23a29cd5ae26f40537da36ab2a463b2825c72d4e05ddb9121938cae762799f3df2a2c8ac590e1308b14d619ebdd123395626bdc7a4b7329daed72f0f148c483206a10d825ff2d7bfa7332ee37380689e3974b912b44c9e59fcd0a897d71bf9c6b1ddb7c99c225b77b4101087a36a4947a64231982326bb4b3cfcad1074cf36beb5bf0dbd224f69908787ecdbdae199c9934987bce93ac07be4739abbc78e16e228eebcc61547798c5edcdd9eda768e962c080287c4585ec95ca'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# importing necessary libraries
import warnings
warnings.filterwarnings("ignore")
# ignoring the warnings

In [ ]:
df = pd.read_csv('/kaggle/input/house-price-india-dataset/House Price India.csv')
# importing the dataset and saving it in df dataframe
df

In [ ]:
df.info()
# getting the info of the dataset
# getting type of column and non-null value counts

In [ ]:
df.describe()
# getting count, mean and standard deviation of each column

In [ ]:
df.corr()


In [ ]:
df.columns
# getting the columns list in the dataset

# DATA VISUALIZATION


In [ ]:
plt.figure(figsize=(17,12),dpi=200)
sns.heatmap(df.corr(),annot=True);

# creating a heatmap to see the distribution

In [ ]:
sns.scatterplot(data=df,x='Built Year',y='Renovation Year')
# visualizing the built year vs renovation year of the house

In [ ]:
sns.countplot(data=df,x='number of bedrooms')


In [ ]:
sns.barplot(data=df,x='Price')

In [ ]:
sns.scatterplot(data=df,x='number of bedrooms',y='lot area',hue='Price')

## DATA PREPROCESSING AND REGULARIZATION

In [ ]:
# checking null values
df.isnull().sum()
# there is no null values

In [ ]:
df['number of bathrooms']

In [ ]:

df.rename(columns = {'number of bathrooms':'no_of_bathrooms'}, inplace = True)

In [ ]:
# rounding off the values of number of bathrooms to next place
df.no_of_bathrooms = df.no_of_bathrooms.round()

In [ ]:
df['no_of_bathrooms']
# the values are rounded off

In [ ]:
# we will use a countplot to visualize it
sns.countplot(data=df,x='no_of_bathrooms')

In [ ]:
# We don't need id and date present in the dataset
df.drop(['id','Date'],axis=1,inplace=True)

In [ ]:
df

In [ ]:
# labelling X and y values
X = df.drop('Price',axis=1)
y = df['Price']

In [ ]:
# splitting the Dataset to train and test data
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
...     X, y, test_size=0.30, random_state=101)
# splitting in 70 to 30 percent dataset

In [ ]:
# standardizing the data
# importing standardscaler for regularization

from sklearn.preprocessing import StandardScaler
# creating scaler instance
scaler = StandardScaler()

In [ ]:
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## Model Creation and Evaluation

In [ ]:
# Creating a linear Regression model
from sklearn.linear_model import LinearRegression

In [ ]:
model = LinearRegression()

In [ ]:
# fitting X_train and y_train to the model
model.fit(X_train,y_train)

In [ ]:
# getting the prediciton
y_pred = model.predict(X_test)

In [ ]:
y_pred

In [ ]:
# evaluating the model
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
# getting mean absolute error
mae = mean_absolute_error(y_test,y_pred)
mae

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test,y_pred))
#getting root mean squared error
rmse

## Creating Elasticnet model and evaluating the model

In [ ]:
# importing elastic net
from sklearn.linear_model import ElasticNet
el_model = ElasticNet()

In [ ]:
para = {'alpha':[0.1,1,5,10,100],'l1_ratio':[.1,.5,.7,.95,.99,1]}

In [ ]:
# using grid search to get the best hyperparameters for our model
from sklearn.model_selection import GridSearchCV

In [ ]:
grid_model = GridSearchCV(estimator=el_model,param_grid=para,
                         scoring='neg_mean_absolute_error',
                         cv = 5,verbose =1)

In [ ]:
# fitting the grid model to find best parameter
grid_model.fit(X_train,y_train)

In [ ]:
grid_model.best_params_
# getting the best hyperparameters

In [ ]:
y_new_pred = grid_model.predict(X_test)

In [ ]:
# calculating the mean absolute error
mae = mean_absolute_error(y_test,y_new_pred)
mae

In [ ]:
# calculating the root mean squared error
rmse = np.sqrt(mean_squared_error(y_test,y_new_pred))
rmse

In [ ]:
# getting the model score for linear Regression
model.score(X_test,y_test)

In [ ]:
# Creating Elastic model with best hyperparameters
# hyperparameters were choosed by using Grid Search
el_model =ElasticNet(alpha=1,l1_ratio=0.7)

In [ ]:
el_model.fit(X_train,y_train)

In [ ]:
pred = el_model.predict(X_test)
# predicting the model

In [ ]:
mae = mean_absolute_error(y_test,pred)
mae
# getting the mean absolute error

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test,pred))
rmse
# getting root mean squared error

In [ ]:
el_model.score(X_test,y_test)
# getting the model score for the elastic model